Always good practice in the beginning: Where am I, what's in my folder?

In [ ]:
!pwd

In [48]:
# without the ! also works in most cases
!ls

'Customer Analysis Round 1 - solution.ipynb'   files_for_lab/   readme.md


In [2]:
import pandas as pd
import numpy as np

# Load all three files into dataframes

In [3]:
df1 = pd.read_csv('lab-customer-analysis-round-1/files_for_lab/csv_files/file1.csv')
df2 = pd.read_csv('lab-customer-analysis-round-1/files_for_lab/csv_files/file2.csv')
df3 = pd.read_csv('lab-customer-analysis-round-1/files_for_lab/csv_files/file3.csv')

# Show all shapes of the dataframes

In [4]:
[df.shape for df in (df1, df2, df3)]

[(4008, 11), (996, 11), (7070, 11)]

In [5]:
# or one by one:

print(df1.shape)
print(df2.shape)
print(df3.shape)

(4008, 11)
(996, 11)
(7070, 11)


# Standardize header names

First check, what's wrong with them

In [6]:
df1.columns.to_list()

['Customer',
 'ST',
 'GENDER',
 'Education',
 'Customer Lifetime Value',
 'Income',
 'Monthly Premium Auto',
 'Number of Open Complaints',
 'Policy Type',
 'Vehicle Class',
 'Total Claim Amount']

column names are not consistently in lower or upper case and also seperated by whitespaces. We need to get rid of whitespaces and make them all lower case.

Problem: We need to have the **same** job done for **several** things.
Solution:

    * Step 1: We get a solution for 1 case (1 column name)
    * Step 2: We scale up and apply that solution to all column names
    
Good problem solving approach in data analytics in general! Keep it in mind!

Let's make it work for one column. We write a function for that:

In [7]:
def standardize_col(col):
    return col.lower().replace(" ", "_")

Let's try it for the string `Total Claim Amount`

In [8]:
standardize_col("Total Claim Amount")

'total_claim_amount'

works fine, so lets apply it to all columns

In [9]:
for col in df1.columns.to_list():
    print(standardize_col(col))

customer
st
gender
education
customer_lifetime_value
income
monthly_premium_auto
number_of_open_complaints
policy_type
vehicle_class
total_claim_amount


In [10]:
# now we create a list were we store all these standardized
# columns in
stdzd_cols = []
for col in df1.columns.to_list():
    stdzd_cols.append(standardize_col(col))

and then overwrite these columns with the original columns

In [11]:
df1.columns = stdzd_cols

In [12]:
# checking whether change was effective
df1.columns

Index(['customer', 'st', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount'],
      dtype='object')

now, we need to perfom this job on two other dataframes. So why not write a function that receives a dataframe and standardizes the columns

In [12]:
def standardize_df_colnames(df):
    #this is just copy pasted from above
    stdzd_cols = []
    for col in df.columns.to_list():
        # making use of the standardize_col function from above
        stdzd_cols.append(standardize_col(col))
    # in the end, overwrite columns with standardized ones
    df.columns = stdzd_cols

In [13]:
# standardize df2
standardize_df_colnames(df2)

# standardize df3
standardize_df_colnames(df3)


In [14]:
df2.columns

Index(['customer', 'st', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'total_claim_amount', 'policy_type', 'vehicle_class'],
      dtype='object')

In [15]:
df3.columns

Index(['customer', 'state', 'customer_lifetime_value', 'education', 'gender',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'total_claim_amount', 'vehicle_class'],
      dtype='object')

#### column naming mismatch 

notice that there is a different naming convention for one of the data sources, state <> st 

+ i recommend that you sort out the naming of the column in the original data frame, to make life easier!


In [16]:
df3.rename(columns={'state':'st'}, inplace=True)

### Why do we standardize?

* it's more convenient working with a dataframe for which you don't have to remember exactly which column name is capital / lower case letters
* sometimes you see that columns are not refered to as `df3['vehicle_class']` but `df3.vehicle_class`. The latter would not be possible if you dont have the `_` between the words

# Concatenate dataframes

How should we concatenate the dataframes? Stick them together horizontally (column wise) or vertically (row wise). Let's look at them first

In [17]:
df1.head()

customer          st gender             education customer_lifetime_value  \
0  RB50392  Washington    NaN                Master                     NaN   
1  QZ44356     Arizona      F              Bachelor              697953.59%   
2  AI49188      Nevada      F              Bachelor             1288743.17%   
3  WW63253  California      M              Bachelor              764586.18%   
4  GA49547  Washington      M  High School or Below              536307.65%   

    income  monthly_premium_auto number_of_open_complaints     policy_type  \
0      0.0                1000.0                    1/0/00   Personal Auto   
1      0.0                  94.0                    1/0/00   Personal Auto   
2  48767.0                 108.0                    1/0/00   Personal Auto   
3      0.0                 106.0                    1/0/00  Corporate Auto   
4  36357.0                  68.0                    1/0/00   Personal Auto   

   vehicle_class  total_claim_amount  
0  Four-Door Car            2.704934  
1  Four-Door Car         1131.464935  
2   Two-Door Car          566.472247  
3            SUV          529.881344  
4  Four-Door Car           17.269323

In [18]:
df3.head()

customer          st  customer_lifetime_value             education gender  \
0  SA25987  Washington              3479.137523  High School or Below      M   
1  TB86706     Arizona              2502.637401                Master      M   
2  ZL73902      Nevada              3265.156348              Bachelor      F   
3  KX23516  California              4455.843406  High School or Below      F   
4  FN77294  California              7704.958480  High School or Below      M   

   income  monthly_premium_auto  number_of_open_complaints    policy_type  \
0       0                   104                          0  Personal Auto   
1       0                    66                          0  Personal Auto   
2   25820                    82                          0  Personal Auto   
3       0                   121                          0  Personal Auto   
4   30366                   101                          2  Personal Auto   

   total_claim_amount  vehicle_class  
0          499.200000   Two-Door Car  
1            3.468912   Two-Door Car  
2          393.600000  Four-Door Car  
3          699.615192            SUV  
4          484.800000            SUV

In [19]:
df2.head()

customer          st gender education customer_lifetime_value  income  \
0  GS98873     Arizona      F  Bachelor              323912.47%   16061   
1  CW49887  California      F    Master              462680.11%   79487   
2  MY31220  California      F   College              899704.02%   54230   
3  UH35128      Oregon      F   College             2580706.30%   71210   
4  WH52799     Arizona      F   College              380812.21%   94903   

   monthly_premium_auto number_of_open_complaints  total_claim_amount  \
0                    88                    1/0/00               633.6   
1                   114                    1/0/00               547.2   
2                   112                    1/0/00               537.6   
3                   214                    1/1/00              1027.2   
4                    94                    1/0/00               451.2   

      policy_type  vehicle_class  
0   Personal Auto  Four-Door Car  
1    Special Auto            SUV  
2   Personal Auto   Two-Door Car  
3   Personal Auto     Luxury Car  
4  Corporate Auto   Two-Door Car

column wise it is! We don't need to think about ordering the column names of the individual data frames. Pandas takes care of that when concatenating

In [20]:
df1.head()

customer          st gender             education customer_lifetime_value  \
0  RB50392  Washington    NaN                Master                     NaN   
1  QZ44356     Arizona      F              Bachelor              697953.59%   
2  AI49188      Nevada      F              Bachelor             1288743.17%   
3  WW63253  California      M              Bachelor              764586.18%   
4  GA49547  Washington      M  High School or Below              536307.65%   

    income  monthly_premium_auto number_of_open_complaints     policy_type  \
0      0.0                1000.0                    1/0/00   Personal Auto   
1      0.0                  94.0                    1/0/00   Personal Auto   
2  48767.0                 108.0                    1/0/00   Personal Auto   
3      0.0                 106.0                    1/0/00  Corporate Auto   
4  36357.0                  68.0                    1/0/00   Personal Auto   

   vehicle_class  total_claim_amount  
0  Four-Door Car            2.704934  
1  Four-Door Car         1131.464935  
2   Two-Door Car          566.472247  
3            SUV          529.881344  
4  Four-Door Car           17.269323

In [21]:
# we concatenate rowwise. We don't need to change "axis" parameter,
# because axis=0 (rowwise) is the default
df_all = pd.concat([df1, df2, df3],
                  # axis=0,
                  )

In [22]:
df_all

customer          st gender             education  \
0     RB50392  Washington    NaN                Master   
1     QZ44356     Arizona      F              Bachelor   
2     AI49188      Nevada      F              Bachelor   
3     WW63253  California      M              Bachelor   
4     GA49547  Washington      M  High School or Below   
...       ...         ...    ...                   ...   
7065  LA72316  California      M              Bachelor   
7066  PK87824  California      F               College   
7067  TD14365  California      M              Bachelor   
7068  UP19263  California      M               College   
7069  Y167826  California      M               College   

     customer_lifetime_value   income  monthly_premium_auto  \
0                        NaN      0.0                1000.0   
1                 697953.59%      0.0                  94.0   
2                1288743.17%  48767.0                 108.0   
3                 764586.18%      0.0                 106.0   
4                 536307.65%  36357.0                  68.0   
...                      ...      ...                   ...   
7065             23405.98798  71941.0                  73.0   
7066             3096.511217  21604.0                  79.0   
7067             8163.890428      0.0                  85.0   
7068             7524.442436  21941.0                  96.0   
7069             2611.836866      0.0                  77.0   

     number_of_open_complaints     policy_type  vehicle_class  \
0                       1/0/00   Personal Auto  Four-Door Car   
1                       1/0/00   Personal Auto  Four-Door Car   
2                       1/0/00   Personal Auto   Two-Door Car   
3                       1/0/00  Corporate Auto            SUV   
4                       1/0/00   Personal Auto  Four-Door Car   
...                        ...             ...            ...   
7065                         0   Personal Auto  Four-Door Car   
7066                         0  Corporate Auto  Four-Door Car   
7067                         3  Corporate Auto  Four-Door Car   
7068                         0   Personal Auto  Four-Door Car   
7069                         0  Corporate Auto   Two-Door Car   

      total_claim_amount  
0               2.704934  
1            1131.464935  
2             566.472247  
3             529.881344  
4              17.269323  
...                  ...  
7065          198.234764  
7066          379.200000  
7067          790.784983  
7068          691.200000  
7069          369.600000  

[12074 rows x 11 columns]

Now, we need to reset the index, because the index numbers are still from the original dataframe (check e.g. that we have 12074 rows, but our index ends at 7069)

In [23]:
df_all = df_all.reset_index(drop=True)
# or do (same effect)
# df_all.reset_index(drop=True, inplace=True)

In [24]:
df_all

customer          st gender             education  \
0      RB50392  Washington    NaN                Master   
1      QZ44356     Arizona      F              Bachelor   
2      AI49188      Nevada      F              Bachelor   
3      WW63253  California      M              Bachelor   
4      GA49547  Washington      M  High School or Below   
...        ...         ...    ...                   ...   
12069  LA72316  California      M              Bachelor   
12070  PK87824  California      F               College   
12071  TD14365  California      M              Bachelor   
12072  UP19263  California      M               College   
12073  Y167826  California      M               College   

      customer_lifetime_value   income  monthly_premium_auto  \
0                         NaN      0.0                1000.0   
1                  697953.59%      0.0                  94.0   
2                 1288743.17%  48767.0                 108.0   
3                  764586.18%      0.0                 106.0   
4                  536307.65%  36357.0                  68.0   
...                       ...      ...                   ...   
12069             23405.98798  71941.0                  73.0   
12070             3096.511217  21604.0                  79.0   
12071             8163.890428      0.0                  85.0   
12072             7524.442436  21941.0                  96.0   
12073             2611.836866      0.0                  77.0   

      number_of_open_complaints     policy_type  vehicle_class  \
0                        1/0/00   Personal Auto  Four-Door Car   
1                        1/0/00   Personal Auto  Four-Door Car   
2                        1/0/00   Personal Auto   Two-Door Car   
3                        1/0/00  Corporate Auto            SUV   
4                        1/0/00   Personal Auto  Four-Door Car   
...                         ...             ...            ...   
12069                         0   Personal Auto  Four-Door Car   
12070                         0  Corporate Auto  Four-Door Car   
12071                         3  Corporate Auto  Four-Door Car   
12072                         0   Personal Auto  Four-Door Car   
12073                         0  Corporate Auto   Two-Door Car   

       total_claim_amount  
0                2.704934  
1             1131.464935  
2              566.472247  
3              529.881344  
4               17.269323  
...                   ...  
12069          198.234764  
12070          379.200000  
12071          790.784983  
12072          691.200000  
12073          369.600000  

[12074 rows x 11 columns]

Let's insert a data cleaning step here already and delete all the rows that contain **only** `NaN` values. Because what's the point in having them? :-)

In [25]:
# creates you a boolean mask for every row that has NaN values only! check also .any() etc...
df_all.isna().all(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
12069    False
12070    False
12071    False
12072    False
12073    False
Length: 12074, dtype: bool

filter the rows by applying this "sieve" to the dataframe. We're using `~` here because we want to negate / invert the boolean "sieve" and **keep** the rows where it says `False`

In [26]:
df_all = df_all[~df_all.isna().all(axis=1)]

In [27]:
df_all = df_all.reset_index(drop=True)

## drop duplicate rows

In [28]:
df_all = df_all.drop_duplicates()

## filter negative incomes

because we regard those as data errors

In [29]:
df_all = df_all[df_all['income'] >= 0]

# Which columns are numerical?

See [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html)

In [30]:
df_all.select_dtypes(np.number)

income  monthly_premium_auto  total_claim_amount
0         0.0                1000.0            2.704934
1         0.0                  94.0         1131.464935
2     48767.0                 108.0          566.472247
3         0.0                 106.0          529.881344
4     36357.0                  68.0           17.269323
...       ...                   ...                 ...
9132  71941.0                  73.0          198.234764
9133  21604.0                  79.0          379.200000
9134      0.0                  85.0          790.784983
9135  21941.0                  96.0          691.200000
9136      0.0                  77.0          369.600000

[9134 rows x 3 columns]

or even more convenient

In [31]:
df_nums = df_all._get_numeric_data()

# Which are categorical?

Short answer? The rest :-)

In [32]:
# drop all numerical columns
df_cats = df_all.drop(columns=df_nums.columns)

In [33]:
df_cats

customer          st gender             education  \
0     RB50392  Washington    NaN                Master   
1     QZ44356     Arizona      F              Bachelor   
2     AI49188      Nevada      F              Bachelor   
3     WW63253  California      M              Bachelor   
4     GA49547  Washington      M  High School or Below   
...       ...         ...    ...                   ...   
9132  LA72316         NaN      M              Bachelor   
9133  PK87824         NaN      F               College   
9134  TD14365         NaN      M              Bachelor   
9135  UP19263         NaN      M               College   
9136  Y167826         NaN      M               College   

     customer_lifetime_value number_of_open_complaints     policy_type  \
0                        NaN                    1/0/00   Personal Auto   
1                 697953.59%                    1/0/00   Personal Auto   
2                1288743.17%                    1/0/00   Personal Auto   
3                 764586.18%                    1/0/00  Corporate Auto   
4                 536307.65%                    1/0/00   Personal Auto   
...                      ...                       ...             ...   
9132                   23406                         0   Personal Auto   
9133                 3096.51                         0  Corporate Auto   
9134                 8163.89                         3  Corporate Auto   
9135                 7524.44                         0   Personal Auto   
9136                 2611.84                         0  Corporate Auto   

      vehicle_class       state  
0     Four-Door Car         NaN  
1     Four-Door Car         NaN  
2      Two-Door Car         NaN  
3               SUV         NaN  
4     Four-Door Car         NaN  
...             ...         ...  
9132  Four-Door Car  California  
9133  Four-Door Car  California  
9134  Four-Door Car  California  
9135  Four-Door Car  California  
9136   Two-Door Car  California  

[9134 rows x 9 columns]

# Understand the meaning

Usually: Check the documentation of the study

# Perform the data cleaning operations mentioned so far in class

Data we have
* `df_nums`
* `df_cats`

Let's look what's dirty here. Caution: We don't want to drop rows after we split into dataframes of categorical and numerical columns because later, after data cleaning, when we stitch them back together, we might end up with mismatching row lengths.

In [34]:
# check for number of null values in every column
df_cats.isna().sum()

customer                        0
st                           7070
gender                        122
education                       0
customer_lifetime_value         7
number_of_open_complaints       0
policy_type                     0
vehicle_class                   0
state                        2064
dtype: int64

or with percentages

In [35]:
df_cats.isna().sum()/len(df_cats)

customer                     0.000000
st                           0.774031
gender                       0.013357
education                    0.000000
customer_lifetime_value      0.000766
number_of_open_complaints    0.000000
policy_type                  0.000000
vehicle_class                0.000000
state                        0.225969
dtype: float64

same with numerics:

In [36]:
df_nums.isna().sum()/len(df_nums)

income                  0.0
monthly_premium_auto    0.0
total_claim_amount      0.0
dtype: float64

Numerics are clean of null values!

### 'st'

In [37]:
df_cats['st'].value_counts()

Oregon        623
California    486
Arizona       327
Nevada        223
Washington    181
Cali          120
AZ             74
WA             30
Name: st, dtype: int64

The states column `st` has a lot of null values (77%) we need to keep that in mind. We have the feeling though that it might turn out as an important column, since the origin of our customers might be good predictors (we'll learn about that later in the week) for other things!

### 'gender'

In [38]:
df_cats['gender'].value_counts()

F         4557
M         4368
Male        40
female      30
Femal       17
Name: gender, dtype: int64

* let's clean all the entries that are not `F` or `M` and turn it into either `F` or `M`

In [39]:
# a task like this can only be solved manually
# a good approach here is a dictionary, because, what do we want to do? Translating!

trans_dct = {'Male': 'M',
             'female':'F',
             'Femal':'F',
             'F':'F',
             'M':'M',
            }

# now, we can write ourselve a function, that does the translate
# and use df_cats['gender'].apply(my_function)
# but we can also make use of the map function, which is way faster (although speed is not 
# important at this point yet)!
df_cats['gender'] = df_cats['gender'].map(trans_dct)

In [40]:
df_cats['gender'].value_counts()

F    4604
M    4408
Name: gender, dtype: int64

BAM! Clean!

### 'education'

In [41]:
df_cats['education'].value_counts()

Bachelor                2718
College                 2681
High School or Below    2616
Master                   751
Doctor                   344
Bachelors                 24
Name: education, dtype: int64

### 'customer_lifetime_value'

In [42]:
df_cats['customer_lifetime_value'].value_counts()

8879.790017           6
4686.469674           6
16473.16905           6
3954.3445340000003    6
9095.049868           6
                     ..
2497.9100129999997    1
9448.833931000001     1
2451.535793           1
9452.996411           1
231138.21%            1
Name: customer_lifetime_value, Length: 8211, dtype: int64

* they're all meant as percent
* but some come with % at the end
* we're casting the values as str first
* then replace all %'s with nothing
* turn the result into float
* and finally multiply by 100

In [43]:
def clean_clv(x):
    return float(str(x).replace('%', ''))*100

df_cats['customer_lifetime_value'] = df_cats['customer_lifetime_value'].apply(clean_clv)

### 'number of open complaints'

In [44]:
df_cats['number_of_open_complaints'].value_counts()

0         5629
1/0/00    1623
1          765
2          283
1/1/00     247
3          230
4          119
1/2/00      93
1/3/00      60
5           44
1/4/00      29
1/5/00      12
Name: number_of_open_complaints, dtype: int64

We see that we have either values of 0, 1, 2, 3, 4 or 5. But then also `1/0/00`, `1/1/00`, `1/2/00`, `1/3/00`, `1/4/00` and `1/5/00`, which contain only redundant info except the middle number. So we create ourselves again a trans dict and roll like above:

In [45]:
trans_dct = {0:0,
             1:1,
             2:2,
             3:3,
             4:4,
             5:5,
             '1/0/00' :0, 
             '1/1/00' :1, 
             '1/2/00' :2, 
             '1/3/00' :3, 
             '1/4/00' :4, 
             '1/5/00' :5, 
            }

df_cats['number_of_open_complaints'] = df_cats['number_of_open_complaints'].map(trans_dct)

In [46]:
df_cats['number_of_open_complaints'].value_counts()

0    7252
1    1012
2     376
3     290
4     148
5      56
Name: number_of_open_complaints, dtype: int64

Clean!